In [17]:
from datetime import date
import pandas as pd
import yaml
from sqlalchemy import create_engine

### Database connections

In [ ]:
# Cargar configuración desde el archivo config.yml
with open('../config.yml', 'r') as f:
    config = yaml.safe_load(f)
    config_fuente = config['fuente']
    config_bodega = config['bodega']

In [ ]:
# Crear conexiones a las bases de datos de fuente y bodega
url_fuente = f"postgresql://{config_fuente['user']}:{config_fuente['password']}@{config_fuente['host']}:{config_fuente['port']}/{config_fuente['dbname']}"
url_bodega = f"postgresql://{config_bodega['user']}:{config_bodega['password']}@{config_bodega['host']}:{config_bodega['port']}/{config_bodega['dbname']}"

# Crear SQLAlchemy engine
fuente_conn = create_engine(url_fuente)
bodega_conn = create_engine(url_bodega)

# Extract

### SQL query

In [ ]:
# Definir consulta SQL para extraer datos de sedes
query = """
SELECT 
    sede.sede_id AS id_sede,
    sede.nombre AS nombre_sede,
    sede.direccion AS Direccion_sede,
    sede.nombre AS ciudad_sede,
    departamento.nombre AS Departamento_sede
FROM sede 
JOIN ciudad ON sede.ciudad_id = ciudad.ciudad_id
JOIN departamento ON ciudad.departamento_id = departamento.departamento_id

"""

In [ ]:
# Leer datos de la fuente en un DataFrame de pandas
dim_sedes = pd.read_sql(query, fuente_conn)


# Transformation

In [22]:
#Agregar fecha de carga
dim_sedes['saved'] = date.today()

### Verifications

In [24]:
# Verificaciones adicionales
print("\nNúmero total de sedes:", len(dim_sedes))
print("\nMuestra de los datos:")
print(dim_sedes.head())


Número total de sedes: 52

Muestra de los datos:
   id_sede            nombre_sede               direccion_sede  \
0       10        FARALLONES /123  Los angeles distrito Latino   
1       11          REMEDIOZ/ 123  Los angeles distrito Latino   
2       13       DIME / LOS ROJOS  Los angeles distrito Latino   
3       14  DESPACHOS / LOS ROJOS  Los angeles distrito Latino   
4       23  POPAYAN BODEGA 28 / A  Los angeles distrito Latino   

             ciudad_sede departamento_sede       saved  
0        FARALLONES /123   VALLE DEL CAUCA  2024-11-09  
1          REMEDIOZ/ 123   VALLE DEL CAUCA  2024-11-09  
2       DIME / LOS ROJOS   VALLE DEL CAUCA  2024-11-09  
3  DESPACHOS / LOS ROJOS   VALLE DEL CAUCA  2024-11-09  
4  POPAYAN BODEGA 28 / A             CAUCA  2024-11-09  


# Load

In [23]:
# Cargar en la bodega de datos
dim_sedes.to_sql('dim_sede', bodega_conn, if_exists='replace', index_label='key_dim_sede')

52